In [1]:
# !pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.2 MB/s eta 0:00:00


In [1]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
import numpy as np
import csv
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, Embedding, Flatten
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
# import keras_tuner as kt
import sklearn as sk
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
import torch 
# import torch.optim as optim
# from sklearn.model_selection import GridSearchCV

In [2]:
def read_data(filename):
    df = pd.read_csv(filename, dtype='float64')
    fieldnames = list(df.columns)
    data = np.array(df.to_dict(orient='records'))
    return data, fieldnames
data, fieldnames = read_data('dataset/rl_data_final_cont.csv')

In [3]:
print('All Fieldnames:', [i for i in enumerate(fieldnames)])

All Fieldnames: [(0, 'bloc'), (1, 'icustayid'), (2, 'charttime'), (3, 'gender'), (4, 'age'), (5, 'elixhauser'), (6, 're_admission'), (7, 'died_in_hosp'), (8, 'died_within_48h_of_out_time'), (9, 'mortality_90d'), (10, 'delay_end_of_record_and_discharge_or_death'), (11, 'Weight_kg'), (12, 'GCS'), (13, 'HR'), (14, 'SysBP'), (15, 'MeanBP'), (16, 'DiaBP'), (17, 'RR'), (18, 'SpO2'), (19, 'Temp_C'), (20, 'FiO2_1'), (21, 'Potassium'), (22, 'Sodium'), (23, 'Chloride'), (24, 'Glucose'), (25, 'BUN'), (26, 'Creatinine'), (27, 'Magnesium'), (28, 'Calcium'), (29, 'Ionised_Ca'), (30, 'CO2_mEqL'), (31, 'SGOT'), (32, 'SGPT'), (33, 'Total_bili'), (34, 'Albumin'), (35, 'Hb'), (36, 'WBC_count'), (37, 'Platelets_count'), (38, 'PTT'), (39, 'PT'), (40, 'INR'), (41, 'Arterial_pH'), (42, 'paO2'), (43, 'paCO2'), (44, 'Arterial_BE'), (45, 'Arterial_lactate'), (46, 'HCO3'), (47, 'mechvent'), (48, 'Shock_Index'), (49, 'PaO2_FiO2'), (50, 'median_dose_vaso'), (51, 'max_dose_vaso'), (52, 'input_total'), (53, 'input_4

In [4]:
all_features = fieldnames[3:7] + fieldnames[11:50] + [fieldnames[56]] + fieldnames[57:59] + fieldnames[50:52] + fieldnames[59:61]
#features = ['Weight_kg', 'GCS', 'HR', 'SysBP', 'MeanBP', 'DiaBP', 'RR', 'SpO2', 'Temp_C', 'FiO2_1', 'Potassium', 'Sodium', 'Chloride', 'Glucose', 'BUN', 'Creatinine', 'Magnesium', 'Calcium', 'Ionised_Ca', 'CO2_mEqL', 'SGOT', 'SGPT', 'Total_bili', 'Albumin', 'Hb', 'WBC_count', 'Platelets_count', 'PTT', 'PT', 'INR', 'Arterial_pH', 'paO2', 'paCO2', 'Arterial_BE', 'Arterial_lactate', 'HCO3', 'mechvent', 'Shock_Index', 'PaO2_FiO2', 'cumulated_balance', 'SOFA', 'SIRS', 'median_dose_vaso', 'max_dose_vaso', 'vaso_input', 'iv_input']
print(f'All Features({len(all_features)}):', [i for i in enumerate(all_features)])

All Features(50): [(0, 'gender'), (1, 'age'), (2, 'elixhauser'), (3, 're_admission'), (4, 'Weight_kg'), (5, 'GCS'), (6, 'HR'), (7, 'SysBP'), (8, 'MeanBP'), (9, 'DiaBP'), (10, 'RR'), (11, 'SpO2'), (12, 'Temp_C'), (13, 'FiO2_1'), (14, 'Potassium'), (15, 'Sodium'), (16, 'Chloride'), (17, 'Glucose'), (18, 'BUN'), (19, 'Creatinine'), (20, 'Magnesium'), (21, 'Calcium'), (22, 'Ionised_Ca'), (23, 'CO2_mEqL'), (24, 'SGOT'), (25, 'SGPT'), (26, 'Total_bili'), (27, 'Albumin'), (28, 'Hb'), (29, 'WBC_count'), (30, 'Platelets_count'), (31, 'PTT'), (32, 'PT'), (33, 'INR'), (34, 'Arterial_pH'), (35, 'paO2'), (36, 'paCO2'), (37, 'Arterial_BE'), (38, 'Arterial_lactate'), (39, 'HCO3'), (40, 'mechvent'), (41, 'Shock_Index'), (42, 'PaO2_FiO2'), (43, 'cumulated_balance'), (44, 'SOFA'), (45, 'SIRS'), (46, 'median_dose_vaso'), (47, 'max_dose_vaso'), (48, 'vaso_input'), (49, 'iv_input')]


In [5]:
static_features = fieldnames[3:7]
print(f'Static Features({len(static_features)}):', [i for i in enumerate(static_features)])

Static Features(4): [(0, 'gender'), (1, 'age'), (2, 'elixhauser'), (3, 're_admission')]


In [6]:
labels = fieldnames[3:7] + fieldnames[11:50] + [fieldnames[56]] + fieldnames[57:59]
#labels = ['Weight_kg', 'GCS', 'HR', 'SysBP', 'MeanBP', 'DiaBP', 'RR', 'SpO2', 'Temp_C', 'FiO2_1', 'Potassium', 'Sodium', 'Chloride', 'Glucose', 'BUN', 'Creatinine', 'Magnesium', 'Calcium', 'Ionised_Ca', 'CO2_mEqL', 'SGOT', 'SGPT', 'Total_bili', 'Albumin', 'Hb', 'WBC_count', 'Platelets_count', 'PTT', 'PT', 'INR', 'Arterial_pH', 'paO2', 'paCO2', 'Arterial_BE', 'Arterial_lactate', 'HCO3', 'mechvent', 'Shock_Index', 'PaO2_FiO2', 'cumulated_balance', 'SOFA', 'SIRS']
print(f'labels({len(labels)}):', [i for i in enumerate(labels)])

labels(46): [(0, 'gender'), (1, 'age'), (2, 'elixhauser'), (3, 're_admission'), (4, 'Weight_kg'), (5, 'GCS'), (6, 'HR'), (7, 'SysBP'), (8, 'MeanBP'), (9, 'DiaBP'), (10, 'RR'), (11, 'SpO2'), (12, 'Temp_C'), (13, 'FiO2_1'), (14, 'Potassium'), (15, 'Sodium'), (16, 'Chloride'), (17, 'Glucose'), (18, 'BUN'), (19, 'Creatinine'), (20, 'Magnesium'), (21, 'Calcium'), (22, 'Ionised_Ca'), (23, 'CO2_mEqL'), (24, 'SGOT'), (25, 'SGPT'), (26, 'Total_bili'), (27, 'Albumin'), (28, 'Hb'), (29, 'WBC_count'), (30, 'Platelets_count'), (31, 'PTT'), (32, 'PT'), (33, 'INR'), (34, 'Arterial_pH'), (35, 'paO2'), (36, 'paCO2'), (37, 'Arterial_BE'), (38, 'Arterial_lactate'), (39, 'HCO3'), (40, 'mechvent'), (41, 'Shock_Index'), (42, 'PaO2_FiO2'), (43, 'cumulated_balance'), (44, 'SOFA'), (45, 'SIRS')]


In [7]:
# Just to indicate, not used.
action_names = fieldnames[50:52] + fieldnames[59:61]
print(f'Action Names({len(action_names)}):', [i for i in enumerate(action_names)])

Action Names(4): [(0, 'median_dose_vaso'), (1, 'max_dose_vaso'), (2, 'vaso_input'), (3, 'iv_input')]


In [8]:
def get_padded_features_and_labels(data):
    _x = []
    _y = []
    current_id = data[0]['icustayid']
    temp_x = []
    temp_y = []
    for i in range(len(data)-1):
        if data[i]['icustayid'] != data[i+1]['icustayid']:
            continue
        if data[i]['icustayid'] != current_id:
            current_id = data[i]['icustayid']
            _x.append(temp_x)
            _y.append(temp_y)
            temp_x = []
            temp_y = []
        temp_x.append([data[i][k] for k in all_features])
        temp_y.append([data[i+1][k] for k in labels])
    if temp_x != []:
        _x.append(temp_x)
        _y.append(temp_y)
    _x = tf.keras.preprocessing.sequence.pad_sequences(_x, padding='pre', dtype='float64', value=0)
    _y = tf.keras.preprocessing.sequence.pad_sequences(_y, padding='pre', dtype='float64', value=0)
    _x = np.array(_x)
    _y = np.array(_y)
    _x = _x.reshape(_x.shape[0]*_x.shape[1], _x.shape[2])
    _y = _y.reshape(_y.shape[0]*_y.shape[1], _y.shape[2])
    return _x, _y

def get_features_and_labels(data):
    _x = []
    _y = []
    for i in range(len(data)-1):
        if data[i]['icustayid'] != data[i+1]['icustayid']:
            continue
 
        _x.append([data[i][k] for k in all_features])
        _y.append([data[i+1][k] for k in labels])
    return np.array(_x), np.array(_y)

def get_comparison_testing_data(data):
    _X, _y = get_padded_features_and_labels(data)
    _, _X_te, _, _y_te = train_test_split(_X, _y, shuffle=False, test_size=0.2)
    X_te_comp = np.array([i for i in _X_te if not np.array_equal(i, np.zeros(50))])
    y_te_comp = np.array([i for i in _y_te if not np.array_equal(i, np.zeros(46))])
    return X_te_comp, y_te_comp


In [9]:
X, y = get_padded_features_and_labels(data)

print(X.shape, y.shape)

X_tr, X_te, y_tr, y_te = train_test_split(X, y, shuffle=False, test_size=0.2)

X_te_comp, y_te_comp = get_comparison_testing_data(data)

# Normalize the features
scaler_X = sk.preprocessing.StandardScaler().fit(X)
X_tr_scaled = scaler_X.transform(X_tr)
X_te_scaled = scaler_X.transform(X_te)
# X_scaled = scaler_X.transform(X)



# Normalize the labels
scaler_y = sk.preprocessing.StandardScaler().fit(y)
y_tr_scaled = scaler_y.transform(y_tr)
y_te_scaled = scaler_y.transform(y_te)

#print(X_tr_scaled.shape, X_te_scaled.shape, X_scaled.shape)#, y_tr_scaled.shape, y_te_scaled.shape)

(393072, 50) (393072, 46)


Hyperparameter tuning

In [ ]:
def model_builder(config):
    model = TabNetRegressor(optimizer_fn=torch.optim.Adam, 
                            optimizer_params=dict(lr=config["lr"]))
    train_loader=torch.utils.data.DataLoader(X_tr_scaled)
    while True:
        train(model, optimizer, train_loader)  # Train the model
        acc = test(model, test_loader)  # Compute test accuracy
        train.report({"mean_accuracy": acc})  # Report to Tune

    return
    
search_space = {"lr": tune.choice([0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001])}
tuner = tune.Tuner(model_builder, param_space=search_space)
results = tuner.fit()
print(results.get_best_result(metric="score", mode="min").config)

End Tuning

In [10]:

model = TabNetRegressor(optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=0.01))
model.fit(
  X_tr_scaled, y_tr_scaled,
  eval_set=[(X_te_scaled, y_te_scaled)],
    patience=10, 
    batch_size=1024,
    drop_last=False
)
# preds = clf.predict(X_test)

C:\Users\guanc\.conda\envs\tfenv\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.48196 | val_0_mse: 0.32165 |  0:00:29s
epoch 1  | loss: 0.31656 | val_0_mse: 0.28818 |  0:00:54s
epoch 2  | loss: 0.3014  | val_0_mse: 0.28844 |  0:01:18s
epoch 3  | loss: 0.29703 | val_0_mse: 0.2806  |  0:01:42s
epoch 4  | loss: 0.29418 | val_0_mse: 0.27841 |  0:02:07s
epoch 5  | loss: 0.29113 | val_0_mse: 0.27746 |  0:02:31s
epoch 6  | loss: 0.28548 | val_0_mse: 0.27229 |  0:02:56s
epoch 7  | loss: 0.27698 | val_0_mse: 0.2769  |  0:03:20s
epoch 8  | loss: 0.27113 | val_0_mse: 0.29054 |  0:03:44s
epoch 9  | loss: 0.26659 | val_0_mse: 0.26389 |  0:04:06s
epoch 10 | loss: 0.25832 | val_0_mse: 0.25387 |  0:04:31s
epoch 11 | loss: 0.25561 | val_0_mse: 0.34309 |  0:04:56s
epoch 12 | loss: 0.25614 | val_0_mse: 0.25486 |  0:05:20s
epoch 13 | loss: 0.25467 | val_0_mse: 0.24614 |  0:05:44s
epoch 14 | loss: 0.25371 | val_0_mse: 0.26069 |  0:06:08s
epoch 15 | loss: 0.25344 | val_0_mse: 0.25198 |  0:06:33s
epoch 16 | loss: 0.25321 | val_0_mse: 0.25727 |  0:06:57s
epoch 17 | los

C:\Users\guanc\.conda\envs\tfenv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [ ]:
# X_tr_reshaped = X_tr_scaled.reshape((X_tr_scaled.shape[0], 
#                                     1, X_tr_scaled.shape[1]))
# X_te_reshaped = X_te_scaled.reshape((X_te_scaled.shape[0], 
#                                     1, X_te_scaled.shape[1]))

# earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
#                                              mode='min',
#                                              min_delta=0,
#                                              patience=7,
#                                              verbose=1)

# model.fit(X_tr_reshaped, y_tr_scaled, 
#           validation_data=(X_te_reshaped, y_te_scaled), 
#           epochs=100, batch_size=1024, verbose=1, callbacks=[earlystop])
# model.fit(X_tr_scaled, y_tr_scaled)

In [11]:
def get_prediction_mse(model, X, y):
    _X = scaler_X.transform(X)
    _y = scaler_y.transform(y)
    _X_st = _X[:, :4]
#     _X = _X.reshape((_X.shape[0], 1, _X.shape[1]))
    prediction = model.predict(_X)
#     prediction = scaler_y.inverse_transform(prediction)
    prediction[:, :4] = _X_st#[:, :4]
#     prediction = prediction[:, :46]
    error = sk.metrics.mean_squared_error(prediction, _y)
    return error

In [12]:
print(get_prediction_mse(model, X_tr, y_tr))
print(get_prediction_mse(model, X_te, y_te))
print(get_prediction_mse(model, X_te_comp, y_te_comp))

0.2998992331890322
0.23323927069241093
0.35775733877662835


In [13]:
torch.save(model, 'saved_models/tabnet.pt')

In [14]:
temp_model = torch.load('saved_models/tabnet.pt')

In [15]:
print(get_prediction_mse(temp_model, X_tr, y_tr))
print(get_prediction_mse(temp_model, X_te, y_te))
print(get_prediction_mse(temp_model, X_te_comp, y_te_comp))

0.2998992331890322
0.23323927069241093
0.35775733877662835
